In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
datasets = ['ADULT', 'German', 'Lawschool', 'HealthHeritage']  # set to include only the datasets on which you have already obtained the data
random_seed = 42 # set the random seed you ran your experiments on
n_samples = 50

networks = {'Linear': 'linear', 'FC NN': 'fc', 'FC NN Large': 'fc_large', 'CNN (BN)': 'cnn', 'ResNet (BN)': 'residual'}

In [ ]:
dataframes_per_dataset = {}

for dataset_name in datasets:
    
    dataset_dataframe = pd.DataFrame()
    dataset_dataframe['Attack'] = ['Inverting Gradients', 'GradInversion', 'TabLeak']
    
    data_per_network = np.zeros((3, 5), dtype='object')
    
    # GradInversion loading
    for j, (network_name, network_load_name) in enumerate(networks.items()):
        
        n_iters = 1500 if network_load_name in ['linear', 'fc', 'fc_large'] else 7000
        
        load_path = f'experiment_data/attacks_with_priors/{dataset_name}/experiment_3/{network_load_name}/batch_size_32/inversion_data_all_3_{dataset_name}_{n_samples}_1_{n_iters}_32_0.319_{random_seed}.npy'
        
        if os.path.isfile(load_path):
            experiment_data = np.load(load_path)[0, :2]
            min_err_idx = np.argmin(experiment_data[0]).item()
            err, std = experiment_data[:, min_err_idx].flatten()
            data_per_network[1, j] = (np.around(100-100*err, 1), np.around(100*std, 1))
        else:
            print(f'Experiment not found, please run ./varying_model_architecture.sh {dataset_name} {n_samples} {random_seed} <max_n_cpus> <start_cpu>')
            break
            
    # Inverting gradients and TabLeak
    for idx, experiment_num in zip([0, 2], [0, 46]):
        
        post_selection = 1 if experiment_num == 0 else 30
        
        # load the linear model and the fully connected networks
        load_path = f'experiment_data/network_size_variation/{dataset_name}/experiment_{experiment_num}/batch_size_32/inversion_data_all_{experiment_num}_{dataset_name}_{n_samples}_{post_selection}_1500_32_0.319_{random_seed}.npy'
        
        if os.path.isfile(load_path):
            experiment_data = np.load(load_path)[0, [0, 3, 5], 0, :2]
            tupled_data = [(np.around(100-100*err, 1), np.around(100*std, 1)) for err, std in experiment_data]
        else:
            print(f'Experiment not found, please run ./varying_model_architecture.sh {dataset_name} {n_samples} {random_seed} <max_n_cpus> <start_cpu>')
            break
        
        # load cnn and resnet
        load_path = f'experiment_data/network_layout_variation/{dataset_name}/experiment_{experiment_num}/batch_size_32/inversion_data_all_{experiment_num}_{dataset_name}_50_{post_selection}_7000_32_0.319_{random_seed}.npy'
        
        if os.path.isfile(load_path):
            experiment_data = np.load(load_path)[0, :, 0, :2]
            tupled_data += [(np.around(100-100*err, 1), np.around(100*std, 1)) for err, std in experiment_data]
        else:
            print(f'Experiment not found, please run ./varying_model_architecture.sh {dataset_name} {n_samples} {random_seed} <max_n_cpus> <start_cpu>')
            break
        
        data_per_network[idx] = tupled_data
    
    for network_name, network_data in zip(networks.keys(), data_per_network.T):
        dataset_dataframe[network_name] = network_data
        
    dataframes_per_dataset[dataset_name] = dataset_dataframe

### Adult

In [ ]:
dataframes_per_dataset['ADULT']

### German

In [ ]:
dataframes_per_dataset['German']

### Lawschool

In [ ]:
dataframes_per_dataset['Lawschool']

### HealthHeritage

In [ ]:
dataframes_per_dataset['HealthHeritage']